In [7]:
# === CELDA DE CONFIGURACIÓN WANDB (EJECUTAR PRIMERO) ===
import wandb
import os

# Pegar tu API key aquí
WANDB_API_KEY = "619223e73ca0d6bb2a62fe224779121e4c7ebe1a"  # Reemplaza con tu API key de wandb

def setup_wandb_online():
    """Configura Weights & Biases en modo online"""
    print("=== Configuración de Weights & Biases Online ===")
    
    # Configurar API key
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    
    # Intentar login
    try:
        wandb.login()
        
        # Verificar configuración con una prueba
        with wandb.init(
            project="butterfly-classification",
            name="test-connection",
            tags=["test"],
        ) as run:
            wandb.log({"connection_test": 1.0})
            
        print("\n✓ Conexión exitosa con wandb")
        print("✓ Login verificado")
        print("✓ Logging probado correctamente")
        print("\nWeights & Biases está configurado correctamente en modo online")
        
    except Exception as e:
        print(f"\n❌ Error al configurar wandb: {e}")
        print("Por favor, verifica tu API key y conexión a internet")
        raise e

# Ejecutar configuración
setup_wandb_online()

=== Configuración de Weights & Biases Online ===



❌ Error al configurar wandb: Run initialization has timed out after 90.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.
Por favor, verifica tu API key y conexión a internet


CommError: Run initialization has timed out after 90.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.

In [2]:
# === CELDA 0: VERIFICACIÓN DEL ENTORNO ===
import sys
import torch
import os

def verify_environment():
    print("Verificando entorno...")
    
    # Verificar Python
    print(f"Python version: {sys.version}")
    
    # Verificar PyTorch y CUDA
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    
    # Verificar estructura del proyecto
    required_files = ['models.py', 'utils.py', 'conf/config.yaml']
    for file in required_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"Missing required file: {file}")
    
    # Verificar dataset
    if not os.path.exists('Butterfly-dataset'):
        raise FileNotFoundError("Dataset directory not found")
        
    print("\nEntorno verificado correctamente ✓")

verify_environment()

Verificando entorno...
Python version: 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]
PyTorch version: 2.5.1+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3070

Entorno verificado correctamente ✓


In [3]:
# ============= CELDA 1: IMPORTS Y CONFIGURACIÓN INICIAL =============
import os
import sys
import torch
import lightning as L
import hydra
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from omegaconf import DictConfig, OmegaConf
from datetime import datetime
from pathlib import Path

# Importar módulos locales
from models import ButterflyAutoencoder, ButterflyClassifier, ButterflyDataModule
from utils import (clean_gpu_memory, verify_dataset_structure, plot_confusion_matrix,
                  measure_inference_performance, plot_training_curves,
                  compare_model_variants, save_experiment_config)

# Configurar warnings y reproducibilidad
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(42)
plt.style.use('seaborn')

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [ ]:
# ============= CELDA 2: DEFINICIÓN DE CONFIGURACIONES =============

# Configuración base del experimento
BASE_CONFIG = {
    'seed': 42,
    'data_dir': 'Butterfly-dataset',
    'batch_size': 32,
    'num_workers': 4,
    'max_epochs': 100,
    'early_stopping_patience': 10,
}

# Configuración para experimento 70/30
CONFIG_70_30 = {
    'name': 'exp1_70_30',
    'description': 'Experiment with 70% unlabeled and 30% labeled data',
    'labeled_ratio': 0.3,
    'autoencoder': {
        'epochs': 50,
        'learning_rate': 1e-3,
        'weight_decay': 1e-5,
        'noise_factor': 0.1,
    },
    'classifier_variants': [
        {
            'name': 'from_scratch',
            'description': 'Classifier trained from scratch',
            'use_pretrained': False,
            'freeze_encoder': False,
            'learning_rate': 1e-3,
            'weight_decay': 1e-5,
        },
        {
            'name': 'pretrained_frozen',
            'description': 'Classifier with frozen pretrained encoder',
            'use_pretrained': True,
            'freeze_encoder': True,
            'learning_rate': 1e-3,
            'weight_decay': 1e-5,
        },
        {
            'name': 'pretrained_unfrozen',
            'description': 'Classifier with unfrozen pretrained encoder',
            'use_pretrained': True,
            'freeze_encoder': False,
            'learning_rate': 1e-4,
            'weight_decay': 1e-5,
        }
    ]
}

# Configuración para experimento 90/10
CONFIG_90_10 = {
    'name': 'exp1_90_10',
    'description': 'Experiment with 90% unlabeled and 10% labeled data',
    'labeled_ratio': 0.1,
    'autoencoder': {
        'epochs': 75,
        'learning_rate': 1e-3,
        'weight_decay': 1e-5,
        'noise_factor': 0.1,
    },
    'classifier_variants': CONFIG_70_30['classifier_variants']  # Mismas variantes
}

# ============= CELDA 3: FUNCIONES DE ENTRENAMIENTO =============

def setup_wandb(config: dict, experiment_name: str, variant_name: str = None):
    """Configura Weights & Biases para el experimento"""
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config")
    
    run_name = f"{experiment_name}_{variant_name}" if variant_name else experiment_name
    try:
        wandb.init(
            project=cfg.wandb.project,
            entity=cfg.wandb.entity,
            name=run_name,
            config={**config, **OmegaConf.to_container(cfg, resolve=True)},
            tags=cfg.wandb.tags + [experiment_name],
            mode=cfg.wandb.mode
        )
    except Exception as e:
        print(f"Warning: Could not initialize wandb: {e}")
        os.environ["WANDB_MODE"] = "offline"

def train_autoencoder(config: dict, datamodule: ButterflyDataModule) -> ButterflyAutoencoder:
    """Entrena el autoencoder"""
    print(f"\nTraining Autoencoder for {config['name']}")
    
    setup_wandb(config, f"autoencoder_{config['name']}")
    
    model = ButterflyAutoencoder(
        learning_rate=config['autoencoder']['learning_rate'],
        weight_decay=config['autoencoder']['weight_decay']
    )
    
    trainer = L.Trainer(
        max_epochs=config['autoencoder']['epochs'],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        callbacks=[
            L.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=BASE_CONFIG['early_stopping_patience'],
                mode='min'
            ),
            L.callbacks.ModelCheckpoint(
                monitor='val_loss',
                mode='min'
            )
        ]
    )
    
    trainer.fit(model, 
               train_dataloaders=datamodule.train_dataloader(labeled=False),
               val_dataloaders=datamodule.val_dataloader())
    
    wandb.finish()
    return model

def train_classifier(config: dict,
                    variant_config: dict,
                    datamodule: ButterflyDataModule,
                    pretrained_encoder: torch.nn.Module = None) -> ButterflyClassifier:
    """Entrena una variante del clasificador"""
    print(f"\nTraining Classifier: {variant_config['name']}")
    
    setup_wandb(config, f"classifier_{config['name']}", variant_config['name'])
    
    model = ButterflyClassifier(
        num_classes=20,
        learning_rate=variant_config['learning_rate'],
        weight_decay=variant_config['weight_decay'],
        pretrained_encoder=pretrained_encoder if variant_config['use_pretrained'] else None,
        freeze_encoder=variant_config['freeze_encoder']
    )
    
    trainer = L.Trainer(
        max_epochs=BASE_CONFIG['max_epochs'],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        callbacks=[
            L.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=BASE_CONFIG['early_stopping_patience'],
                mode='min'
            ),
            L.callbacks.ModelCheckpoint(
                monitor='val_loss',
                mode='min'
            )
        ]
    )
    
    trainer.fit(model,
               train_dataloaders=datamodule.train_dataloader(labeled=True),
               val_dataloaders=datamodule.val_dataloader())
    
    # Evaluación
    test_results = trainer.test(model, datamodule.test_dataloader())[0]
    wandb.log({"test_results": test_results})
    
    # Matriz de confusión
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in datamodule.test_dataloader():
            x, y = batch
            logits = model(x)
            preds = torch.argmax(logits, dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    plot_confusion_matrix(
        y_true,
        y_pred,
        datamodule.get_class_names(),
        title=f"Confusion Matrix - {variant_config['name']}"
    )
    
    # Análisis de rendimiento
    perf_metrics = measure_inference_performance(model, datamodule.test_dataloader())
    wandb.log(perf_metrics)
    
    wandb.finish()
    return model, test_results, perf_metrics

# ============= CELDA 4: FUNCIÓN PRINCIPAL DE EXPERIMENTO =============

def run_experiment(config: dict):
    """Ejecuta un experimento completo"""
    print(f"\nStarting experiment: {config['name']}")
    print(f"Description: {config['description']}")
    
    # Crear directorio para resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = Path(f"outputs/{config['name']}_{timestamp}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Verificar dataset
    if not verify_dataset_structure(BASE_CONFIG['data_dir']):
        raise ValueError("Dataset structure is invalid")
    
    # Inicializar DataModule
    datamodule = ButterflyDataModule(
        data_dir=BASE_CONFIG['data_dir'],
        batch_size=BASE_CONFIG['batch_size'],
        num_workers=BASE_CONFIG['num_workers'],
        labeled_ratio=config['labeled_ratio']
    )
    datamodule.setup()
    
    # Entrenar autoencoder
    autoencoder = train_autoencoder(config, datamodule)
    clean_gpu_memory()
    
    # Entrenar variantes del clasificador
    results = []
    for variant_config in config['classifier_variants']:
        try:
            model, test_results, perf_metrics = train_classifier(
                config,
                variant_config,
                datamodule,
                pretrained_encoder=autoencoder.encoder if variant_config['use_pretrained'] else None
            )
            
            results.append({
                'variant': variant_config['name'],
                'test_accuracy': test_results['test_acc'],
                'test_f1': test_results['test_f1'],
                **perf_metrics
            })
            
            clean_gpu_memory()
            
        except Exception as e:
            print(f"Error training {variant_config['name']}: {e}")
    
    # Comparar resultados
    compare_model_variants(
        results,
        metrics=['test_accuracy', 'test_f1', 'avg_latency', 'model_size_mb'],
        save_dir=output_dir
    )
    
    # Guardar configuración
    save_experiment_config(config, output_dir)
    
    return results

# ============= CELDA 5: EJECUCIÓN DE EXPERIMENTOS =============

def main():
    """Función principal de ejecución"""
    try:
        # Verificar GPU y configurar reproducibilidad
        print(f"Using device: {device}")
        torch.manual_seed(BASE_CONFIG['seed'])
        
        # Configurar wandb
        if os.environ.get("WANDB_API_KEY") is None:
            print("WANDB: Running in offline mode")
            os.environ["WANDB_MODE"] = "offline"
        
        # Ejecutar experimento 70/30
        print("\nStarting 70/30 experiment...")
        results_70_30 = run_experiment(CONFIG_70_30)
        
        # Preguntar si continuar con 90/10
        response = input("\nContinue with 90/10 experiment? (y/n): ")
        if response.lower() == 'y':
            print("\nStarting 90/10 experiment...")
            results_90_10 = run_experiment(CONFIG_90_10)
        
    except KeyboardInterrupt:
        print("\nExperiment interrupted by user")
    except Exception as e:
        print(f"\nError in execution: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        clean_gpu_memory()
        if wandb.run is not None:
            wandb.finish()

if __name__ == "__main__":
    main()

# ============= CELDA 6 (OPCIONAL): ANÁLISIS DE RESULTADOS =============
# Esta celda se puede ejecutar después de completar los experimentos

def analyze_results():
    """Analiza y compara resultados de ambos experimentos"""
    results_dir = Path("outputs")
    
    # Encontrar los directorios más recientes para cada experimento
    exp_70_30_dir = sorted(results_dir.glob("exp1_70_30_*"))[-1]
    exp_90_10_dir = sorted(results_dir.glob("exp1_90_10_*"))[-1]
    
    # Cargar resultados
    results_70_30 = pd.read_csv(exp_70_30_dir / "model_comparison.csv")
    results_90_10 = pd.read_csv(exp_90_10_dir / "model_comparison.csv")
    
    # Comparar resultados
    plt.figure(figsize=(15, 10))
    
    metrics = ['test_accuracy', 'test_f1']
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 1, i)
        
        x = np.arange(len(results_70_30))
        width = 0.35
        
        plt.bar(x - width/2, results_70_30[metric], width, label='70/30 split')
        plt.bar(x + width/2, results_90_10[metric], width, label='90/10 split')
        
        plt.xlabel('Model Variant')
        plt.ylabel(metric)
        plt.title(f'Comparison of {metric}')
        plt.xticks(x, results_70_30['variant'], rotation=45)
        plt.legend()
    
    plt.tight_layout()
    plt.show()

# Para ejecutar el análisis:
# analyze_results()